Thanks to Russ for providing code from his last scraper: https://www.kaggle.com/sasrdw/gbt5fx

We adapted a few things and rather focused on non-US countries.

This is just a first attempt, and we are happy for others to contribute. Please do not rely on these results as they are and some countries might be completely wrong.

In [ ]:
import os
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, date
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('../input/covid19-global-forecasting-week-5/train.csv')
train = train.fillna("")

In [ ]:
def get_wm(date="2020-05-09"):
    

    if date == "2020-05-08":
        f = "http://web.archive.org/web/20200508190022/https://www.worldometers.info/coronavirus/"
    elif date == "2020-05-07":
        f = "http://web.archive.org/web/20200507190101/https://www.worldometers.info/coronavirus/"
    elif date == "2020-05-06":
        f = "http://web.archive.org/web/20200506190052/https://www.worldometers.info/coronavirus/"
    elif date == "2020-04-27":
        f = "http://web.archive.org/web/20200427190152/https://www.worldometers.info/coronavirus/"
    else:
        f = "https://www.worldometers.info/coronavirus/"
    
    fname = "page.html"
    !wget $f -q -O $fname
    df_list = pd.read_html(open(fname))
    
    cm = {'Country, Territory':'Country_Region', 'Total Cases':'TotalCases', 'New Cases':'NewCases',
      'Total Deaths':'TotalDeaths', 'Total Recovered': 'TotalRecovered', 'Serious, Critical': 'SeriousCritical',
       'Country, Other':'Country_Region', 'Country,Other':'Country_Region'}

    df0 = df_list[0]
    df0.insert(0,'Date',date)
    df0.rename(mapper=cm, axis=1, inplace=True)
    
    inttype = ['NewCases','NewDeaths']
    
    for c in inttype:
        df0[c] = df0[c].astype(str).map(lambda x: x.lstrip('+-,')).str.replace(',', '').astype(float)
        
    df0 = df0[~df0["Country_Region"].isin(["World", "Total:", "North America", "Europe", "Asia", "South America", "Africa", "Oceania",])]
    
    wm = df0
    
    wm["County"] = np.nan
    wm["Province_State"] = np.nan
    
    recode = {
     'CAR': 'Central African Republic',
     'Congo':'Congo (Brazzaville)',
     'Cruise Ship':'Diamond Princess',
     'Curaçao':'Curacao',
     'DRC':'Congo (Kinshasa)',
     'Faeroe Islands':'Faroe Islands',
     'Falkland Islands': 'Falkland Islands (Malvinas)',
     'Ivory Coast':"Cote d'Ivoire",
     'Macao':'Macau',
     'Myanmar':'Burma',
     'Palestine':'West Bank and Gaza',
     'Réunion':'Reunion',
     'S. Korea':'Korea, South',
     'Saint Martin':'St Martin',
     'Saint Pierre Miquelon':'Saint Pierre and Miquelon',
     'St. Vincent Grenadines':'Saint Vincent and the Grenadines',
     'St. Barth':'Saint Barthelemy',
     'Taiwan':'Taiwan*',
     'Turks and Caicos':'Turks and Caicos Islands',
     'U.A.E.':'United Arab Emirates',
     'U.K.':'United Kingdom',
     'U.S. Virgin Islands': 'Virgin Islands',
     'UAE':'United Arab Emirates',
     'UK':'United Kingdom',
     'USA':'US',
     'USA *':'US',
     'Vatican City':'Holy See',
    "Caribbean Netherlands" : "Bonaire, Sint Eustatius and Saba"
    }
    wm['Country_Region'] = wm['Country_Region'].replace(to_replace=recode)
    
    

    c = pd.Series(wm["Country_Region"].unique())
    idx = c.isin(train["Country_Region"])
   # print(idx.sum())
   # print(c[~idx])
    ct = c[~idx].values
    
    for c in ct:
        t = train[train["Province_State"]==c]["Country_Region"].values[0]
        #print(c, "-", t)
        idx = wm["Country_Region"]==c
        wm.loc[idx, "Province_State"] = c
        wm.loc[idx, "Country_Region"] = t
        
    wm["NewCases"] = wm["NewCases"].fillna(0)
    wm["NewDeaths"] = wm["NewDeaths"].fillna(0)
    wm["Province_State"] = wm["Province_State"].fillna("")
    wm["County"] = wm["County"].fillna("")
    
    last_day = train[train.Date==train.Date.max()]
    last_day_c = last_day[last_day.Target=="ConfirmedCases"]
    last_day_f = last_day[last_day.Target=="Fatalities"]
    
    ignore_countries = ["China", "US", "Canada", "Australia", "Kosovo", "Brazil", "Spain"]
        
    rows = []
    for i, row in last_day_c.iterrows():
        #print(row)
        w = wm[(wm["Province_State"]==row["Province_State"]) & (wm["Country_Region"]==row["Country_Region"]) & (wm["County"]==row["County"])]

        if len(w) < 1 or row["Country_Region"] in ignore_countries:
            row["TargetValue"] = np.nan
        else:
            row["TargetValue"] = w["NewCases"].values[0]

        row["Date"] = wm["Date"].values[0]
        rows.append(row)
        
    new_c = pd.DataFrame(rows)
    
    rows = []
    for i, row in last_day_f.iterrows():
        #print(row)
        w = wm[(wm["Province_State"]==row["Province_State"]) & (wm["Country_Region"]==row["Country_Region"]) & (wm["County"]==row["County"])]

        if len(w) < 1 or row["Country_Region"] in ignore_countries:
            row["TargetValue"] = np.nan
        else:
            row["TargetValue"] = w["NewDeaths"].values[0]

        row["Date"] = wm["Date"].values[0]
        rows.append(row)
        
    new_f = pd.DataFrame(rows)
    
    new = pd.concat([new_c, new_f], axis=0)
        
    
    new = new.sort_values(["Country_Region", "Target"])
    
    return new

df = get_wm(date="2020-04-27")

df.head()

In [ ]:
df[df.TargetValue.isna()]["Country_Region"].value_counts()

In [ ]:
train.Date.max()

In [ ]:
#train_new = pd.concat([train, df], axis=0).sort_values(["Country_Region", "Province_State", "County", "Date", "Target"]).reset_index(drop=True)

In [ ]:
train_new = df

In [ ]:
train_new.head()

In [ ]:
train_new[train_new["Country_Region"]=="Afghanistan"].sort_values("Date", ascending=True)

In [ ]:
train_new[train_new.TargetValue.isna()]

In [ ]:
train_new.to_csv("train_extra.csv", index=False)

In [ ]:
train_new = pd.concat([train, df], axis=0).sort_values(["Country_Region", "Province_State", "County", "Date", "Target"]).reset_index(drop=True)

In [ ]:
train_new.to_csv("train.csv", index=False)

In [ ]:
train = train_new.copy()

In [ ]:
#train = train.fillna(0)

In [ ]:
def fix_target(frame, key, target, new_target_name="target"):
    import numpy as np
    group_keys = frame[ key].values.tolist()
    target = frame[target].values.tolist()
    for i in range(1, len(group_keys) - 1):
        target[i] =max(0,target[i] )#correct negative values
    frame[new_target_name] = np.array(target)

directory=""

train_fatalities=train[train.Target=="Fatalities"]
train_confirmed=train[train.Target=="ConfirmedCases"]
train_fatalities.columns=["Id_Fatalities","County","Province_State","Country_Region",
                          "Population","Weight_Fatalities","Date","Target","diff_Fatalities"]
train_confirmed.columns=["Id_ConfirmedCases","County","Province_State","Country_Region",
                         "Population","Weight_ConfirmedCases","Date","Target","diff_ConfirmedCases"]
train_confirmed.drop("Population", inplace=True, axis=1)
train_fatalities.drop("Target", axis=1, inplace=True)
train_confirmed.drop("Target", axis=1, inplace=True)
train=pd.merge(train_confirmed, train_fatalities, how="left", left_on=["County","Province_State","Country_Region","Date"],
              right_on=["County","Province_State","Country_Region","Date"])
train=train[["Id_Fatalities","Id_ConfirmedCases","Date","County","Province_State","Country_Region",
                          "Population","Weight_ConfirmedCases","Weight_Fatalities","diff_ConfirmedCases","diff_Fatalities"]]
train["key"]=train[["County","Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1])+ "_" + str(row[2]),axis=1)

fix_target(train, "key", "diff_ConfirmedCases", new_target_name="diff_ConfirmedCases")
fix_target(train, "key", "diff_Fatalities", new_target_name="diff_Fatalities")

#train["diff_ConfirmedCases"] = train["ConfirmedCases"]

train["ConfirmedCases"] = train.groupby("key")["diff_ConfirmedCases"].transform("cumsum")
train["Fatalities"] = train.groupby("key")["diff_Fatalities"].transform("cumsum")
#get_cumulative(train, "key", "diff_ConfirmedCases", new_target_name="ConfirmedCases")
#get_cumulative(train, "key", "diff_Fatalities", new_target_name="Fatalities")


In [ ]:
train[train["Country_Region"]=="Afghanistan"].sort_values("Date", ascending=False)

In [ ]:
train[train["key"]=="__China"].sort_values("Date", ascending=False)

In [ ]:
train.to_csv("train_oldformat.csv", index=False)